In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

import time
import datetime

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.simplefilter('ignore')

### Подход 1: градиентный бустинг "в лоб"


**Задание 1.**
Считайте таблицу с признаками из файла features.csv. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
# Читаем таблицу с признаками.

data = pd.read_csv('data/features.csv', index_col='match_id')
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_hero,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_hero,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_hero,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,1,11,4,3,1058,996,12,0,0,6,42,4,1085,986,12,0,0,4,21,5,2052,1536,23,0,0,6,37,3,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,0,5,39,5,1960,1384,16,0,0,8,88,3,640,566,1,0,1,5,79,3,720,1350,2,2,0,12,7,2,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,0,9,22,5,2305,2028,19,1,1,10,66,3,1024,959,19,0,1,10,86,3,755,620,3,0,0,8,29,4,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,0,6,96,5,1878,1174,17,0,0,6,48,3,732,1468,22,0,0,10,15,4,1681,1051,11,0,0,7,102,2,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,0,8,26,3,704,586,1,0,2,9,69,3,1169,1665,20,1,0,7,22,3,1055,638,1,0,0,9,25,5,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
# Смотрим на количество признаков.

data.shape

(97230, 108)

In [4]:
# Определяем столбцы с финальными результатами и удаляем.

result_cols = ['duration', 
               'radiant_win', 
               'tower_status_radiant', 
               'tower_status_dire',
               'barracks_status_radiant',
               'barracks_status_dire']

X = data.drop(result_cols, axis=1)
X.shape

(97230, 102)

**Задание 2.**
Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
# Как видно, пропуски есть в нескольких признаках (их количество меньше размера таблицы - 97230).

X.count().sort_values().head(15)

first_blood_player2            53243
radiant_flying_courier_time    69751
dire_flying_courier_time       71132
first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
dire_bottle_time               81087
radiant_bottle_time            81539
radiant_first_ward_time        95394
dire_first_ward_time           95404
radiant_courier_time           96538
dire_courier_time              96554
d3_hero                        97230
d4_xp                          97230
d4_level                       97230
dtype: int64

In [6]:
# Ниже приведен список признаков, в которых есть пропущенные значения.

X.columns[(X.count() < X.shape[0])].tolist()

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

**Задание 3.** Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [7]:
# Заполняем пропуски нулями

X.fillna(0, inplace=True)

**Задание 4.** Какой столбец содержит целевую переменную? Запишите его название.

In [8]:
# Выделим ответы

y = data['radiant_win']

**Задание 5.**
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [9]:
# Разбиения для кросс-валидации

cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
# Кросс-валидация с подбором оптимального n_estimators с помощью GridSearchCV
# Зафиксировал random_state, чтобы потом сравнить качество для другой реализации
# Время работы измеряем выставлением параметра verbose=1

param_grid = {
    'n_estimators': [10, 20, 30, 50, 100]
}

clf = GradientBoostingClassifier(random_state=42)
clfcv_1 = GridSearchCV(clf, param_grid, scoring='roc_auc', verbose=1, cv=cv)
clfcv_1.fit(X, y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  6.4min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 20, 30, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [11]:
# Лучшее качество по кросс-валидации и соответствующие параметры

clfcv_1.best_score_, clfcv_1.best_params_

(0.7062229783991215, {'n_estimators': 100})

In [12]:
# Кросс-валидация для n_estimators=30 для ответа на вопрос

param_grid = {
    'n_estimators': [30]
}

clf = GradientBoostingClassifier(random_state=42)
clfcv_2 = GridSearchCV(clf, param_grid, scoring='roc_auc', verbose=1, cv=cv)
clfcv_2.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   59.4s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [30]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=1)

In [13]:
# Качество по кросс-валидации с 30 деревьями

clfcv_2.best_score_

0.6900064710388156

In [14]:
# Оценим качество по кросс-валидации с измерением времени для разного количества деревьев
# Используем другую реализацию - с помощью cross_val_score и измерением времени работы кода, как в задании

for n_estimators in [10, 20, 30, 50, 100]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators, random_state=42)
    score = cross_val_score(clf, X, y, scoring='roc_auc', cv=cv)
    print('Trees: {}\tAUC: {}\tTime elapsed: {}'.format(n_estimators,
                                                        score.mean(),
                                                        datetime.datetime.now() - start_time))

Trees: 10	AUC: 0.6648506879750012	Time elapsed: 0:00:21.286990
Trees: 20	AUC: 0.6824618768044435	Time elapsed: 0:00:37.395006
Trees: 30	AUC: 0.6900064710388155	Time elapsed: 0:00:56.565994
Trees: 50	AUC: 0.6974943609466162	Time elapsed: 0:01:34.670865
Trees: 100	AUC: 0.7062229783991214	Time elapsed: 0:03:33.443101


In [15]:
# Видно, что качество на 30 деревьях получилось аналогичным, как и при использовании GridSearchCV, 
# что и следовало ожидать при одинаковом random_state

**Вопросы и ответы:**

1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
    - Список признаков с пропусками: 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time'.
    - Из описания признаков события "первая кровь" (first blood) известно, что если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение. То есть, для четырёх первых признаков выполнилось условие, что в первые 5 минут не было события "первая кровь", отсюда и пропуски в данных.    
2. Как называется столбец, содержащий целевую переменную?
    - Наша задача - по первым 5 минутам игры предсказать, какая из команд победит. Соответственно, целевая переменная - radiant_win (1 - победила команда Radiant, 0 — иначе).
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?
    - Время: 0:00:56.565994
    - Качество: 0.6900064710388155
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
    - Есть смысл использовать больше 30 деревьев. При обучении обычно фиксируют число деревьев и подбирают 'learning_rate' и другие параметры. Для ускорения обучения можно ограничивать глубину деревьев 'max_depth' или использовать случайное подмножество исходной выборки.

### Подход 2: логистическая регрессия

**Задание 1.**
Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [16]:
# Масштабируем признаки

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [17]:
# Оценим качество по кросс-валидации

def cv_logreg(X, y):
    param_grid = {'C': np.logspace(-5, 0, 10)}
    logreg = LogisticRegression(penalty='l2', solver='lbfgs')
    return GridSearchCV(logreg, param_grid, scoring='roc_auc', verbose=1, cv=cv).fit(X, y)

In [18]:
logreg_1 = cv_logreg(X_scaled, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   44.3s finished


In [19]:
# Лучший параметр регуляризации

logreg_1.best_estimator_.C

0.005994842503189409

In [20]:
# Лучшее качество

logreg_1.best_score_

0.716555049539802

**Задание 2.**
Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [21]:
# Выделим категориальные признаки

hero_cols = [col for col in X.columns if '_hero' in col]
cat_cols = ['lobby_type'] + hero_cols

ix = np.isin(X.columns, cat_cols)
X_scaled_num = X_scaled[:, ~ix]

In [22]:
# Обучим заново модель без категориальных признаков

logreg_2 = cv_logreg(X_scaled_num, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   43.5s finished


In [23]:
logreg_2.best_estimator_.C

0.005994842503189409

In [24]:
# Качество при таком подходе сильно не изменилось

logreg_2.best_score_

0.7165644078390465

**Задание 3.**
На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [25]:
# Определим количество уникальных идентификаторов для каждого героя

X[hero_cols].nunique()

r1_hero    108
r2_hero    108
r3_hero    108
r4_hero    108
r5_hero    108
d1_hero    108
d2_hero    108
d3_hero    108
d4_hero    108
d5_hero    108
dtype: int64

In [26]:
# Так как для каждого героя уникальные идентификаторы могут отличаться, определим их количество для всех героев сразу

hero_nunique = np.unique(X[hero_cols].values.reshape(-1)).shape[0]
hero_nunique

108

In [27]:
# Не все идентификаторы в исследуемых играх задействованы, поэтому определим максимальный задействованный
# Его будем использовать для формирования "мешка слов"

hero_nunique_max = np.unique(X[hero_cols].values.reshape(-1)).max()
hero_nunique_max

112

**Задание 4.**
Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [28]:
# Формируем "мешок слов" по героям

def bag_of_words(data, N):
    def apply_value(indices, value):
        for idx in range(X_pick.shape[0]):
            X_pick[idx, indices[idx]] = value
    X_pick = np.zeros((data.shape[0], N))
    for p in range(1, 6):
        apply_value(data['r%d_hero' % p].values-1, 1)
        apply_value(data['d%d_hero' % p].values-1, -1)
    return X_pick

In [29]:
X_pick = bag_of_words(X, hero_nunique_max)

In [30]:
# Добавим полученные признаки к числовым

X_all = np.hstack((X_scaled_num, X_pick))

**Задание 5.**
Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [31]:
# Обучим модель с категориальными признаками, закодированными "мешком слов"

logreg_3 = cv_logreg(X_all, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.7min finished


In [32]:
logreg_3.best_estimator_.C

0.07742636826811278

In [33]:
# Видно значительный прирост качества, типы героев влияют на вероятность победы команды

logreg_3.best_score_

0.7519485808826895

**Задание 6.**
Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [34]:
data_test = pd.read_csv('data/features_test.csv', index_col='match_id')
data_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_hero,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_hero,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_hero,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,102,3,1183,963,9,0,2,9,52,5,2289,2043,20,2,0,6,53,3,917,1006,14,0,0,6,57,3,1344,933,3,0,0,9,26,3,1122,949,5,1,0,10,89,4,1847,1866,25,1,0,10,20,3,841,1015,2,1,0,9,29,3,1091,918,6,0,1,8,39,5,1674,1048,12,0,1,7,1.0,0.0,2.0,NaN,72.0,-82.0,NaN,2,2,3,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,6,4,1194,1386,21,0,0,7,65,4,1374,1064,13,0,1,8,92,2,774,737,0,1,0,8,43,5,2254,1787,24,0,0,6,54,3,909,970,16,0,0,5,34,4,1543,929,10,0,0,7,93,4,1485,1389,19,0,0,14,99,3,1075,718,3,0,1,7,5,4,1130,994,1,1,0,10,84.0,1.0,9.0,NaN,138.0,-82.0,233.0,2,6,2,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,26,2,421,569,3,0,0,6,29,3,960,896,3,1,2,6,11,5,1877,1511,23,0,0,8,41,4,1581,1593,26,0,0,4,91,3,687,675,1,0,0,1,22,2,515,793,4,0,0,8,10,3,964,1022,12,0,0,8,49,5,2434,1917,22,0,0,9,54,4,1372,2005,24,2,1,11,168.0,1.0,9.0,NaN,33.0,-88.0,NaN,5,4,3,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,91,2,672,901,6,0,0,8,74,5,2141,1626,28,0,0,7,52,4,1579,2169,28,2,0,7,2,5,1960,1668,27,0,0,8,110,4,1255,704,5,0,1,6,25,2,490,648,1,0,1,9,77,4,1308,1355,22,0,0,7,62,2,536,692,0,1,1,6,49,4,1815,1424,18,0,0,5,55.0,0.0,3.0,NaN,208.0,-78.0,180.0,4,2,2,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,93,5,1703,964,10,0,1,6,30,3,1099,942,2,1,0,6,85,3,1285,1013,10,0,0,8,102,2,484,609,4,0,0,8,14,4,2273,1680,16,1,0,6,64,2,511,541,1,0,0,4,70,4,1135,1399,19,0,0,10,3,3,623,520,1,0,0,10,96,5,1884,1409,15,0,1,11,285.0,1.0,5.0,NaN,225.0,-76.0,NaN,1,3,2,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [35]:
# Лучший результат у логистической регрессии из задания 5, поэтому разделим числовые и категориальные признаки, 
# отмасштабируем и сформируем "мешок слов".

# Заполним нулями пропущенные значения
data_test.fillna(0, inplace=True)

# Масштабируем признаки
X_scaled_test = scaler.transform(data_test)

# Выделяем числовые
ix = np.isin(data_test.columns, cat_cols)
X_scaled_num_test = X_scaled_test[:, ~ix]

# Выделяем категориальные и формируем "мешок слов"
X_pick_test = bag_of_words(data_test, hero_nunique_max)

# Объединим числовые и категориальные признаки
X_all_test = np.hstack((X_scaled_num_test, X_pick_test))

In [36]:
# Применим лучшую модель

y_pred = logreg_3.best_estimator_.predict_proba(X_all_test)

In [37]:
# Предсказания модели для каждого класса

y_pred

array([[0.17636985, 0.82363015],
       [0.24740202, 0.75259798],
       [0.81187841, 0.18812159],
       ...,
       [0.76551446, 0.23448554],
       [0.37625688, 0.62374312],
       [0.573391  , 0.426609  ]])

In [38]:
# Минимальное и максимальное значения предсказания

y_pred[:,1].min(), y_pred[:,1].max()

(0.008420548132261197, 0.9963537875772223)

In [39]:
# Результат в файл

data_test['radiant_win'] = y_pred[:, 1]
data_test.to_csv('submission.csv', columns=['radiant_win'])

**Вопросы и ответы:**

1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
    - Качество логистической регрессии над всеми исходными признаками = 0.716555049539802. Качество получилось выше, чем у градиентного бустинга. Это может быть связано с тем, что в бустинге взято малое количество деревьев. Логистическая регрессия работает быстрее бустинга.
2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
    - Удаление категориальных признаков слабо повлияло на результат, качество = 0.7165644078390465. Это может быть связано с тем, что категориальные признаки не были правильным образом подготовлены для логистической регрессии. Так как без обработки категориальный признак воспринимается как вещественное число, но порядковый номер героя никак не связан с его силой в игре.
3. Сколько различных идентификаторов героев существует в данной игре?
    - Всего различных идентификаторов 112.
4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
    - При добавлении "мешка слов" по героям качество = 0.7519485808826895. Оно стало сильно лучше, так как типы героев влияют на вероятность победы команды и теперь сила конкретного героя учитывается в итоговом результате.
5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
    - 0.008420548132261197, 0.9963537875772223